In [1]:
with open('log1125_14.txt', 'r') as f:
    all_logs = f.readlines() 

In [2]:
import re

dataset_list = []
lr_list = []
epochs_list = []
hidden_list = []
dropout_list = []
weight_decay_list = []
alpha_list = []
beta_list = []
gamma_list = []
norm_layers_list = []
orders_list = []
highest_train_list = []
highest_valid_list = []
final_train_list = []
final_test_list = []

for logs in all_logs:
    if 'Namespace' in logs:
        dataset_list.extend(re.findall(r"dataset='(.*?)', directed", logs))
        lr_list.extend(re.findall(r"lr=(.*?), method", logs))
        epochs_list.extend(re.findall(r"epochs=(.*?), gamma", logs))
        dropout_list.extend(re.findall(r"dropout=(.*?), epochs", logs))
        weight_decay_list.extend(re.findall(r"weight_decay=(.*?)\)", logs))
        alpha_list.extend(re.findall(r"alpha=(.*?), beta", logs))
        beta_list.extend(re.findall(r"beta=(.*?), cached", logs))
        orders_list.extend(re.findall(r"orders=(.*?), orders_func_id", logs))
        gamma_list.extend(re.findall(r"gamma=(.*?), gat_heads", logs))
        norm_layers_list.extend(re.findall(r"norm_layers=(.*?), num_layers", logs))
        hidden_list.extend(re.findall(r"hidden_channels=(.*?), hops", logs))
        
    if 'Highest Train' in logs and ',' in logs:
        highest_train_list.extend(re.findall(r"Highest Train: (.*?),", logs))
    
    if 'Highest Valid' in logs and ',' in logs:
        highest_valid_list.extend(re.findall(r"Highest Valid: (.*?),", logs))
        
    if 'Final Train' in logs and ',' in logs:
        final_train_list.extend(re.findall(r"Final Train: (.*?),", logs))
        
    if 'Final Test' in logs and ',' in logs:
        final_test_list.extend(re.findall(r"Final Test: (.*?),", logs))
        

In [3]:
import pandas as pd
import os

d = {
    'dataset': dataset_list,
    'lr': lr_list,
    'epoch': epochs_list,
    'hidden': hidden_list,
    'dropout': dropout_list,
    'weight_decay': weight_decay_list,
    'alpha': alpha_list,
    'beta': beta_list,
    'gamma': gamma_list,
    'norm_layers': norm_layers_list,
    'orders': orders_list,
    'high_train': highest_train_list,
    'high_valid': highest_valid_list,
    'final_train': final_train_list,
    'final_test': final_test_list
}

for k, v in d.items():
    print(k, len(v))

csv_name = 'results/large_dataset_tuning14.csv'
df = pd.DataFrame.from_dict(d)
df = df.sort_values(['final_test']).reset_index(drop=True)
if os.path.exists(csv_name):
    df.to_csv(csv_name, mode='a', header=False)
else:
    df.to_csv(csv_name)
df

dataset 270
lr 270
epoch 270
hidden 270
dropout 270
weight_decay 270
alpha 270
beta 270
gamma 270
norm_layers 270
orders 270
high_train 270
high_valid 270
final_train 270
final_test 270


,dataset,lr,epoch,hidden,dropout,weight_decay,alpha,beta,gamma,norm_layers,orders,high_train,high_valid,final_train,final_test
0,snap-patents,0.01,200,128,0.0,0.0,0.0,0.0,0.0,1,1,19.13,19.15,19.13,19.12
1,snap-patents,0.01,200,128,0.0,0.0,1.0,0.1,1.0,1,1,19.13,19.15,19.13,19.12
2,snap-patents,0.01,200,128,0.0,0.0,0.0,0.1,1.0,1,2,19.13,19.15,19.13,19.12
3,snap-patents,0.01,200,128,0.0,0.0,1.0,0.1,1.0,1,2,19.13,19.15,19.13,19.12
4,snap-patents,0.01,200,128,0.0,0.0,0.0,0.1,1.0,1,3,19.13,19.15,19.13,19.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,snap-patents,0.01,200,128,0.0,0.0,0.0,1.0,0.0,2,3,35.97,35.95,35.97,35.84
266,snap-patents,0.01,200,128,0.0,0.0,0.0,10.0,0.5,3,3,36.20,35.90,36.20,35.86
267,snap-patents,0.01,200,128,0.0,0.0,1.0,10.0,0.0,3,2,37.75,37.73,37.75,37.68
268,snap-patents,0.01,200,128,0.0,0.0,1.0,1.0,0.0,2,3,38.75,38.78,38.75,38.73
